In [1]:
################
################   Check pipeline steps
################   BNP Paribas Cardif Claims Management Kaggle Competition
################
%load_ext autoreload

%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from scipy import stats

# import other methods
from data_modifier import *

##### upload data
train = pd.read_csv("../../../github_data/bnp_paribas_cardif_data/train.csv")
test = pd.read_csv("../../../github_data/bnp_paribas_cardif_data/test.csv")
sample = pd.read_csv("../../../github_data/bnp_paribas_cardif_data/sample_submission.csv")
trains = train[:100]
tests = test[:100]

In [3]:
##### split and clean data
## define variables
y = trains.target
columns = train.columns
x_train = trains[columns[2:]]
x_test = trains[columns[1:]]

## change null to nan
nton = NulltoNanTrans()
nton = nton.fit(x_train)
x_tr_ntont = nton.transform(x_train)
x_te_ntont = nton.transform(x_test)

NullToNaNTrans fit done.
NullToNaNTrans transform done.
NullToNaNTrans transform done.


In [4]:
#### continous data
dat = DataSpliterTrans(dtype=np.float64)
dat = dat.fit(x_tr_ntont)
x_tr_datct = dat.transform(x_tr_ntont)
x_te_datct = dat.transform(x_te_ntont)

## if nan change nan to median
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp = imp.fit(x_tr_datct)
x_tr_ntovt = imp.transform(x_tr_datct)
x_te_ntovt = imp.transform(x_te_datct)

DataSpliterTrans fit done.
DataSpliterTrans transform done.
DataSpliterTrans transform done.


In [5]:
#### categorical data
### int categorical data
dat = DataSpliterTrans(dtype=np.int)
dat = dat.fit(x_tr_ntont)
x_tr_datit = dat.transform(x_tr_ntont)
x_te_datit = dat.transform(x_te_ntont)

## if nan change nan to most frequent
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp = imp.fit(x_tr_datit)
x_tr_ntovt2 = imp.transform(x_tr_datit)
x_te_ntovt2 = imp.transform(x_te_datit)

## transform each category to binary
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc = enc.fit(x_tr_ntovt2)
x_tr_catobit = enc.transform(x_tr_ntovt2)
x_te_catobit = enc.transform(x_te_ntovt2)

DataSpliterTrans fit done.
DataSpliterTrans transform done.
DataSpliterTrans transform done.


In [6]:
### object categorical data
dat = DataSpliterTrans(dtype=np.object)
dat = dat.fit(x_tr_ntont)
x_tr_datot = dat.transform(x_tr_ntont)
x_te_datot = dat.transform(x_te_ntont)

DataSpliterTrans fit done.
DataSpliterTrans transform done.
DataSpliterTrans transform done.


In [7]:
## change object to categorical & convert categories str to int
cat = ObjtoCatStrtoIntTrans()
cat = cat.fit(x_tr_datot)
x_tr_catot = cat.transform(x_tr_datot)
x_tr_catot = cat.transform(x_te_datot)

ObjtoCatStrtoIntTrans fit done.
(100, 19)
ObjtoCatStrtoIntTrans transform done.
(100, 19)
ObjtoCatStrtoIntTrans transform done.


In [8]:
## if nan change nan to most frequent
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imp = imp.fit(x_tr_catot)
x_tr_ntovt3 = imp.transform(x_tr_catot)
x_te_ntovt3 = imp.transform(x_tr_catot)

In [ ]:
## transform each category to binary
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc = enc.fit(x_tr_ntovt3)
x_tr_catobit = enc.transform(x_tr_ntovt3)
x_te_catobit = enc.transform(x_te_ntovt3)